In [20]:
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import math
import os
import csv

from tqdm import tqdm

In [5]:
# 固定随机种子，确保作业可以重现，随机数生产期产生的随机数是一致的
def same_seed(seed): 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# 将数据集分割为验证集和训练集
def train_valid_split(data_set, valid_ratio, seed):
    valid_set_size = int(valid_ratio * len(data_set)) 
    # 验证集的大小为 valid_ration*整个的数据集大小
    
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    # 在data_set数据集中按照8：2的比例分割，并且由于确定了generator固定保证了分割过程的可重复性 
    
    return np.array(train_set), np.array(valid_set)

# 对测试集进行预测
def predict(test_loader, model, device):
    model.eval() 
    # 开启验证模式，不会dropout以及会计算移动的batch normalization
    
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())
            # pred会脱离计算图，并且从gpu移动到cpu上
            
    preds = torch.cat(preds, dim=0).numpy()  
    # 将preds沿着第0维拼接起来，并转化为numpy数组
    
    return preds
     

In [7]:
# 定义一个Dataset
class COVID19Dataset(Dataset):
    def __init__(self, x, y=None):
    # 定义了类的初始化，后面创建实例需要传入参数x和y，如果没有就默认为None
    
        if y is None:
            self.y = y
            # self.y是类里面的属性，y为传进来的参数；x和y为元组
            # 元组，列表可以通过索引来访问某个元素 a[x];而字典需要通过key值来访问某个元素 my_dict['']
            
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
    # 根据索引值返回对应的特征和标签
    
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
    # 返回数据集的长度
        return len(self.x)
 # __getitem__(): element = my_instance[some_index]
 # __len__(): len(my_instance)

In [8]:
# 定义一个神经网络模型
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
         
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) 
        # 将其转化为单维度向量
        
        return x

In [9]:
# 将数据集分割为训练集，验证集和测试集；并提取相应的特征和标签
def select_feat(train_data, valid_data, test_data, select_all=True):
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data
    # [:,-1]选取所有行以及最后一列; [:,:-1]选取所有行，以及从第一列到最后一列

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
        # 输出将展示所有特征列的索引，例如 [0, 1, 2, ..., n]，
    else:
        feat_idx = [0,1,2,3,4] 
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid
     

In [16]:
# 定义训练过程
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') 
    # 定义MSE作为损失函数，并返回均方误差平均值

    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7) 
    # 定义了SGD优化器，用来更新参数；momentum=0.7为动量，将考虑之前梯度的 70% 加上当前梯度的 30%

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
    # early_stop_count 初始化为 0，用于记录模型性能未改善的连续轮数
    
    for epoch in range(n_epochs):
        model.train()
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)
        # tqdm is a package to visualize your training progress.
        # tqdm(iterator)用来封装迭代器，是可以在循环的时候显示长进度条
        # train_loader本身为一个dataloader对象，而dataloader本身为一个批量加载数据的迭代器
        # position=0代表进度条在终端顶端，leave=true代表进度条满了之后不消失

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
            # 设置进度条的的描述信息和和后置信息

        mean_train_loss = sum(loss_record)/len(loss_record)

        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) 
            # 如果当前的loss小于所保存的最小loss，则保存模型
            
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
        # 如果模型性能未改善的连续轮数大于规定值，停止训练
        
            print('\nModel is not improving, so we halt the training session.')
            return

In [13]:
# 参数配置
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}


In [14]:
# 定义你自己的数据集

same_seed(config['seed'])
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
# 读取文件数据，将其转化为DataFrame对象，再转化为numpy数组

train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

# 定义dataset
train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# 定义dataloader
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# shuffle代表会打乱数据，每个batch的数据会随机加载；pin_memory代表加速传到gpu

# 可以看到输出train和valid数据集都有89列，是因为最后一列是标签，而train是不包含标签的

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)
number of features: 88


In [21]:
# 开始训练
model = My_Model(input_dim=x_train.shape[1]).to(device)    # 模型在gpu上运行
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.05it/s, loss=103]


Epoch [1/5000]: Train loss: 149.1424, Valid loss: 95.1712
Saving model with loss 95.171...


Epoch [2/5000]: 100%|███████████████████████████████████████████████████████| 10/10 [00:00<00:00, 51.70it/s, loss=94.5]


Epoch [2/5000]: Train loss: 96.1537, Valid loss: 88.8333
Saving model with loss 88.833...


Epoch [3/5000]: 100%|███████████████████████████████████████████████████████| 10/10 [00:00<00:00, 66.92it/s, loss=89.4]


...

Epoch [4998/5000]: Train loss: 1.5809, Valid loss: 1.3477


Epoch [4999/5000]: 100%|████████████████████████████████████████████████████| 10/10 [00:00<00:00, 78.29it/s, loss=1.27]


Epoch [4999/5000]: Train loss: 1.7295, Valid loss: 1.3762


Epoch [5000/5000]: 100%|████████████████████████████████████████████████████| 10/10 [00:00<00:00, 80.97it/s, loss=1.06]


Epoch [5000/5000]: Train loss: 1.2422, Valid loss: 1.3406


In [22]:
# 定义一个将预测结果写入指定文件的函数
def save_pred(preds, file):
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        # csv.writer()创建一个写入器对象，可以将数据写入到csv文件中；fp指定了要写入的文件
        
        writer.writerow(['id', 'tested_positive'])
        # 写入索引和标题
        
        for i, p in enumerate(preds):
            writer.writerow([i, p])

# 测试集验证，并将结果先写入指定文件
model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 167.85it/s]
